## LangSmith

In [284]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Using Language Models

In [ ]:

# sparkllm
os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""
#　此处参考：https://www.xfyun.cn/doc/spark/Web.html
os.environ["IFLYTEK_SPARK_API_URL"] = ""
os.environ["IFLYTEK_SPARK_llm_DOMAIN"] = ""

from langchain_community.chat_models import ChatSparkLLM

model = ChatSparkLLM()

让我们首先直接使用模型。是LangChain “Runnables” 的实例，这意味着它们公开了一个标准接口来与它们进行交互。只需简单地调用模型，我们就可以将消息列表传递给该方法。`ChatModel.invoke`

In [286]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我叫小明")])

AIMessage(content='你好，小明！有什么我可以帮助你的吗？', response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 4, 'completion_tokens': 10, 'total_tokens': 14}}, id='run-4f71c835-449e-46a3-9333-14bf24509361-0')

该模型本身没有任何状态概念。例如，如果您提出后续问题：

In [287]:
model.invoke([HumanMessage(content="你的名字是什么")])

AIMessage(content='您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。', response_metadata={'token_usage': {'question_tokens': 3, 'prompt_tokens': 3, 'completion_tokens': 40, 'total_tokens': 43}}, id='run-0e9bcde0-e6b9-4ef9-b7cf-eec0422797c3-0')

我们可以看到，它没有将之前的对话变成上下文，也无法回答问题。 这带来了糟糕的聊天机器人体验！

为了解决这个问题，我们需要将整个对话历史记录传递到模型中。让我们看看这样做时会发生什么：

In [288]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好，我是小刚"),
        AIMessage(content="你好小刚，请问今天我可以帮你做些什么？"),
        HumanMessage(content="我的名字是什么？"),
    ]
)

AIMessage(content='你刚才告诉我，你的名字是小刚。', response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 19, 'completion_tokens': 10, 'total_tokens': 29}}, id='run-875e266f-b657-4dd4-9a56-0765c6510871-0')

## 消息历史记录

我们可以使用 Message History 类来包装我们的模型并使其有状态。 这将跟踪模型的输入和输出，并将它们存储在某个数据存储中。 然后，未来的交互将加载这些消息，并将它们作为输入的一部分传递到链中。 让我们看看如何使用它！

首先，让我们确保安装 langchain-community ，因为我们将使用其中的集成来存储消息历史记录。

` ! pip install langchain_community`

之后，我们可以导入相关类并设置我们的链，该链包装模型并添加此消息历史记录。这里的一个关键部分是我们传入的函数。此函数应接受并返回 Message History 对象。这用于区分单独的会话，并且在调用新链时应作为配置的一部分传入（我们将展示如何执行此操作）。`get_session_history`,`session_id`,`session_id`

In [321]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

创建一个包含会话ID (session_id) 的配置对象，并在每次调用 RunnableWithMessageHistory 时传递这个配置对象。这样可以确保在处理每个请求时都能使用正确的会话ID进行会话历史记录的管理。

我们可以定义一个配置对象 `config` ，其中包含 `session_id`。然后在每次调用 RunnableWithMessageHistory 时，将这个配置对象传递给它。

In [290]:
config = {"configurable": {"session_id": "abc2"}}

In [291]:

response = with_message_history.invoke(
    [HumanMessage(content="你好我是小刚")],
    config=config,
)

response.content

'你好，小刚！有什么我可以帮助你的吗？'

In [292]:
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么?")],
    config=config,
)

response.content

'根据你之前所说的信息，你的名字是小刚。'

In [293]:
# 为了更清晰的显示对话历史，我们定义一个函数来打印对话历史
def pretty_print_store(store):
    def print_message(message, indent=0):
        prefix = ' ' * indent
        if isinstance(message, str):
            print(f"{prefix}{message}")
        elif isinstance(message, dict):
            for key, value in message.items():
                print(f"{prefix}{key}:")
                print_message(value, indent + 4)
        else:
            print(f"{prefix}{message}")

    for key, history in store.items():
        print(f"Session ID: {key}")
        print("Messages:")
        for message in history.messages:
            print_message(message, 4)
        print()

In [294]:
pretty_print_store(store)

Session ID: abc2
Messages:
    content='你好我是小刚'
    content='你好，小刚！有什么我可以帮助你的吗？' response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 4, 'completion_tokens': 11, 'total_tokens': 15}} id='run-d7d0501a-b368-4f17-a61b-3c495ccc307e-0'
    content='我的名字是什么?'
    content='根据你之前所说的信息，你的名字是小刚。' response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 19, 'completion_tokens': 12, 'total_tokens': 31}} id='run-879ea651-67f8-494a-b1d6-4ee56105cbb8-0'



我们的聊天机器人现在记住了关于我们的事情。如果我们更改配置以引用不同的 ，我们可以看到它重新开始对话。`session_id`

In [295]:
config = {"configurable": {"session_id": "abc3"}}

In [296]:
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么")],
    config=config,
)

response.content

'很抱歉，我不知道您的名字。请问您可以告诉我您的名字吗？'

In [297]:
pretty_print_store(store)

Session ID: abc2
Messages:
    content='你好我是小刚'
    content='你好，小刚！有什么我可以帮助你的吗？' response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 4, 'completion_tokens': 11, 'total_tokens': 15}} id='run-d7d0501a-b368-4f17-a61b-3c495ccc307e-0'
    content='我的名字是什么?'
    content='根据你之前所说的信息，你的名字是小刚。' response_metadata={'token_usage': {'question_tokens': 4, 'prompt_tokens': 19, 'completion_tokens': 12, 'total_tokens': 31}} id='run-879ea651-67f8-494a-b1d6-4ee56105cbb8-0'

Session ID: abc3
Messages:
    content='我的名字是什么'
    content='很抱歉，我不知道您的名字。请问您可以告诉我您的名字吗？' response_metadata={'token_usage': {'question_tokens': 3, 'prompt_tokens': 3, 'completion_tokens': 13, 'total_tokens': 16}} id='run-77997f7f-8757-49c2-bca4-f1692ec6dd53-0'



但是，我们总是可以回到原始对话（因为我们将其保存在store字典中）

In [298]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么")],
    config=config,
)

response.content

'根据前面你所说过的信息，你的名字是小刚。'

## Prompt templates

提示模板有助于将原始用户信息转换为 LLM 可以使用的格式。在这种情况下，原始用户输入只是一条消息，我们将它传递给 LLM。现在让我们让它更复杂一点。首先，让我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，除了消息之外，我们还将添加更多输入。

首先，让我们添加一条系统消息。为此，我们将创建一个 `ChatPromptTemplate`。我们将利用所有消息传递。`MessagesPlaceholder`

[ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html):是一个用于创建和管理聊天提示的模板类。它允许你定义一个模板，其中包含变量占位符，这些占位符可以在实际使用时被动态值替换。这样可以方便地创建复杂的聊天提示，而不需要手动拼接字符串。

[MessagePlaceholder](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html)：是一种特殊的占位符，用于在聊天提示模板中插入消息历史记录。它允许你在生成新的聊天提示时，自动包含之前的对话内容，从而保持上下文的连续性。


In [299]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个有力的帮手，可以尽你所能帮助我。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

请注意，这略微改变了输入类型 - 我们现在不是传入消息列表，而是传入一个带有键的字典，其中包含消息列表。`messages`

In [300]:
response = chain.invoke({"messages": [HumanMessage(content="你好，我是鲍勃")]})

response.content

'你好，鲍勃！很高兴认识你。请问有什么我可以帮助你的吗？'

现在，我们可以将其包装在与以前相同的 Messages History 对象中

In [301]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [302]:
config = {"configurable": {"session_id": "abc5"}}

In [303]:
response = with_message_history.invoke(
    [HumanMessage(content="你好，我是鲍勃")],
    config=config,
)

response.content

'你好，鲍勃！很高兴为你提供帮助。请问有什么我可以帮助你的？'

In [304]:
response = with_message_history.invoke(
    [HumanMessage(content="我的名字是谁？")],
    config=config,
)

response.content

'你告诉我的，你的名字是鲍勃。'

现在让我们的提示稍微复杂一点。让我们假设提示模板现在看起来像这样：

In [305]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个有帮助的助手。请用 {language} 回答所有问题，并尽你所能提供帮助。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [306]:
response = chain.invoke(
    {"messages": [HumanMessage(content="你好，我是鲍勃")], "language": "英语"}
)

response.content

'你好，鲍勃！很高兴见到你。有什么我可以帮助你的吗？'

现在，让我们将这个更复杂的链包装在 Message History 类中。这一次，由于输入中有多个键，我们需要指定用于保存聊天记录的正确键。

In [307]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [308]:
config = {"configurable": {"session_id": "abc11"}}

In [309]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

## 管理对话历史记录

构建聊天机器人时要了解的一个重要概念是如何管理对话历史记录。如果不进行管理，消息列表将无限制增长，并可能溢出 LLM 的上下文窗口。因此，添加一个步骤来限制您传入的消息的大小非常重要。

重要的是，您需要在提示模板之前执行此操作，但在从消息历史记录加载以前的消息之后执行此操作。

为此，我们可以在提示符前面添加一个简单的步骤来适当地修改密钥，然后将该新链包装在 Message History 类中。messages

LangChain带有一些内置的帮助程序，用于管理消息列表。在本例中，我们将使用 trim
_messages 帮助程序来减少向模型发送的消息数。修剪器允许我们指定要保留的令牌数量，以及其他参数，例如我们是否要始终保留系统消息以及是否允许部分消息：

In [310]:
from langchain_core.messages import SystemMessage
from langchain_core.messages.utils import trim_messages


trimmer = trim_messages(
    max_tokens=60, # 指定修剪后的消息列表中最多包含 60 个令牌。
    strategy="last", # 使用 "last" 策略，这意味着保留最后的消息，直到达到令牌限制。
    token_counter=model, # 指定用于计算消息中令牌数量的函数或模型。这里假设 model 是一个可以计算令牌数量的对象或函数。
    include_system=True, # 保留系统消息，即使它不在修剪策略范围内，也会包含在修剪后的消息列表中。
    allow_partial=False, # 不允许部分消息，这意味着要么保留整个消息，要么完全删除它。
    start_on="human", # 从第一个出现的 HumanMessage 开始修剪。这意味着在修剪过程中，只有在遇到 HumanMessage 之后的消息才会被考虑。
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="please help me with a math problem : what 1+(3*2)-4 is"),
    AIMessage(content="3"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!'),
 HumanMessage(content='please help me with a math problem : what 1+(3*2)-4 is'),
 AIMessage(content='3')]

API 参考：[SystemMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) | [trim_messages](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.utils.trim_messages.html)

要在我们的链中使用它，我们只需要在将输入传递到提示符之前运行修剪器。messages

现在，如果我们尝试向模型询问我们的名字，它不会知道它，因为我们修剪了聊天记录的那部分：

In [311]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, as an AI language model, I don't have the ability to know your name. Is there anything else I can help you with?"

但是，如果我们询问最后几条消息中的信息，它会记住：

In [312]:
print( messages + [HumanMessage(content="what math problem did i ask")])

[SystemMessage(content="you're a good assistant"), HumanMessage(content="hi! I'm bob"), AIMessage(content='hi!'), HumanMessage(content='I like vanilla ice cream'), AIMessage(content='nice'), HumanMessage(content='thanks'), AIMessage(content='no problem!'), HumanMessage(content='having fun?'), AIMessage(content='yes!'), HumanMessage(content='please help me with a math problem : what 1+(3*2)-4 is'), AIMessage(content='3'), HumanMessage(content='what math problem did i ask')]


In [313]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我们刚才的对话内容是什么")],
        "language": "English",
    }
)
response.content

'您刚才问了一个数学问题，问题是 "1+(3*2)-4" 的结果是多少。'

现在让我们将其包装在消息历史记录中

In [323]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [315]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but as an AI language model, I don't have access to your personal information or identity. Is there anything else I can help you with?"

不出所料，我们说出我们名字的第一条消息已被修剪。此外，聊天记录中现在还有两条新消息（我们的最新问题和最新回复）。这意味着过去在我们的对话历史记录中可以访问的更多信息不再可用！在本例中，我们最初的数学问题也从历史记录中删减了，因此模型不再知道它：

In [316]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't have access to your previous queries or conversations. Can you please provide me with the math problem you are referring to? I'll do my best to help you solve it."

## Stream

现在我们有了一个功能聊天机器人。然而，聊天机器人应用程序的一个非常重要的用户体验考虑因素是流媒体。LLM 有时可能需要一段时间才能做出响应，因此为了改善用户体验，大多数应用程序所做的一件事就是在生成每个令牌时流回每个令牌。这允许用户查看进度。

这其实非常容易做到！

所有链都公开一个方法，使用消息历史记录的链也不例外。我们可以简单地使用该方法来返回流式响应。`.stream`

In [324]:
config = {"configurable": {"session_id": "abc14"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Sure, Todd! Here's a joke for you:

Why did the tomato turn red?
Because it saw the salad dressing!|